In [1]:
import pandas as pd
import re

In [2]:
#load download.geonames.org/export/zip/allCountries.zip

data_path='dataset/'
geonames = pd.read_csv(data_path+'allCountries.txt', sep="\t", header=None)
coords = geonames[[0,9,10]].copy()
coords[9]  = [int(x*10)/10 for x in coords[9]]
coords[10] = [int(x*10)/10 for x in coords[10]]
coords=coords.applymap(str)
keys=coords[9]+'$'+coords[10]
values=coords[0]
check=dict(zip(keys,values))

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,4,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#load data
data=pd.read_json("dataset/business.json", lines=True)
data=data[["business_id", "name", "city", "state", "postal_code", "latitude", "longitude", "categories"]].applymap(str)
columns=data.columns

#drop 'nan'
miss_nan=data[data['latitude']=='nan']
data=data.drop(data[data['latitude']=='nan'].index)
print('Dropped: '+str(len(miss_nan)))
miss_nan[:3]

Dropped: 1


,business_id,name,city,state,postal_code,latitude,longitude,categories
136097,W1x0rlzGUrMBbK3Hq5bk2Q,Bixi Bike,Toronto,ON,M5H 4G1,nan,nan,"['Bikes', 'Sporting Goods', 'Shopping']"


In [4]:
# imputiere

keys=data['postal_code']
values=data['latitude']+'$'+data['longitude']
impute=dict(zip(keys,values))

impute_lat=[]
impute_long=[]
for miss in miss_nan['postal_code']:
    try:
        miss=impute[miss]
        impute_lat.append(miss.split('$')[0])
        impute_long.append(miss.split('$')[1])
    except:
        impute_lat.append('')
        impute_long.append('')
miss_nan['latitude']=impute_lat
miss_nan['longitude']=impute_long

miss=miss_nan[miss_nan['latitude']!='nan']
data=data.append(miss)

print('Imputiert: '+str(len(miss)))
miss[:3]

Imputiert: 1


,business_id,name,city,state,postal_code,latitude,longitude,categories
136097,W1x0rlzGUrMBbK3Hq5bk2Q,Bixi Bike,Toronto,ON,M5H 4G1,43.6507644773,-79.38611269,"['Bikes', 'Sporting Goods', 'Shopping']"


In [5]:
data['intlat']  = [str(int(float(x)*10)/10) for x in data['latitude']]
data['longlat'] = [str(int(float(x)*10)/10) for x in data['longitude']]
data['key']=data['intlat']+'$'+data['longlat']
data[:3]

,business_id,name,city,state,postal_code,latitude,longitude,categories,intlat,longlat,key
0,FYWN1wneV18bWNgQjJ2GNg,Dental by Design,Ahwatukee,AZ,85044,33.3306902,-111.9785992,"['Dentists', 'General Dentistry', 'Health & Me...",33.3,-111.9,33.3$-111.9
1,He-G7vWjzVUysIKrfNbPUQ,Stephen Szabo Salon,McMurray,PA,15317,40.2916853,-80.1048999,"['Hair Stylists', 'Hair Salons', ""Men's Hair S...",40.2,-80.1,40.2$-80.1
2,KQPW8lFf1y5BT2MxiSZ3QA,Western Motor Vehicle,Phoenix,AZ,85017,33.5249025,-112.1153098,"['Departments of Motor Vehicles', 'Public Serv...",33.5,-112.1,33.5$-112.1


In [6]:
#drop missing postal
miss_zip=data[data['postal_code']=='']
data=data.drop(data[data['postal_code']==''].index)
print('Dropped: '+str(len(miss_zip)))
miss_zip[:3]

Dropped: 623


,business_id,name,city,state,postal_code,latitude,longitude,categories,intlat,longlat,key
628,m2qv-mG--aQ-bh7iNl1vgg,Swept Away,Phoenix,AZ,,33.4483771,-112.0740373,"['Home Services', 'Home Cleaning']",33.4,-112.0,33.4$-112.0
898,RBOXMNb_gzdmy1CRPxv_4A,The Abbey Bar & Restaurant,Main Street,CO,,51.8124379933,-8.3926353604,"['Irish', 'Bars', 'Nightlife', 'Restaurants']",51.8,-8.3,51.8$-8.3
1160,oIoq6WQbyRpPZClMkHQNlQ,Delicia's Latina's,Las Vegas,NV,,36.1600037195,-115.118577965,"['Restaurants', 'Cuban', 'Mexican', 'Food', 'F...",36.1,-115.1,36.1$-115.1


In [7]:
# imputiere

keys=data['key']
values=data['postal_code']
impute=dict(zip(keys,values))

impute_zip=[]
for miss in miss_zip['key']:
    try:
        miss=impute[miss]
        impute_zip.append(miss)
    except:
        impute_zip.append('')
miss_zip['postal_code']=impute_zip

miss=miss_zip[miss_zip['postal_code']!='']
data=data.append(miss)

print('Imputiert: '+str(len(miss)))
miss[:3]

Imputiert: 609


,business_id,name,city,state,postal_code,latitude,longitude,categories,intlat,longlat,key
628,m2qv-mG--aQ-bh7iNl1vgg,Swept Away,Phoenix,AZ,85016,33.4483771,-112.0740373,"['Home Services', 'Home Cleaning']",33.4,-112.0,33.4$-112.0
1160,oIoq6WQbyRpPZClMkHQNlQ,Delicia's Latina's,Las Vegas,NV,89101,36.1600037195,-115.118577965,"['Restaurants', 'Cuban', 'Mexican', 'Food', 'F...",36.1,-115.1,36.1$-115.1
1233,A-vb-4ck67bw2yATWfmlTA,Premium Cabinets,Gilbert,AZ,85224,33.3393677,-111.8208054,"['Cabinetry', 'Home & Garden', 'Home Services'...",33.3,-111.8,33.3$-111.8


In [8]:
# Daten-Abgleich mit geonames

country=[]
for key in data['key']:
    try:
        country.append(check[key])
    except:
        country.append('')
data['country']=country
print('Fehlende: '+str(len(data[data['country']==''])))

Fehlende: 2932


In [9]:
miss=data[data['country']==''].copy()
data=data[data['country']!=''].copy()

In [10]:
codes=geonames[[0,1,2,3,4,5]].applymap(str)
codes.columns=['country','postal_code','district','state','state_short','county']

In [11]:
# quick handle
codes['key']=codes['postal_code']+'$'+codes['state_short']+'$'+codes['district']
keys=codes['key'].tolist()
values=codes['country'].tolist()
check=dict(zip(keys,values))

codes['short']=codes['postal_code']+'$'+codes['state_short']
keys=codes['short'].tolist()
short=dict(zip(keys,values))

codes['short2']=codes['postal_code']+'$'+codes['district']
keys=codes['short2'].tolist()
short2=dict(zip(keys,values))

def check_zip(postal, state, city):
    postal=str(postal).strip().split(' ')[0]  
    
    #quick
    try:
        return check[postal+'$'+state+'$'+city]
    
    #quick
    except:
        try:
            return short[postal+'$'+state]
        #quick
        except:
            try:
                return short2[postal+'$'+city]
            except:  
                return ''
            
            #super-slow
            """
            match=codes[codes['postal_code']==postal]
            match=match[match['state_short']==state]
            match=match[match['district'].str.contains(city)]
            match=list(set(match['country'].tolist()))
            l=len(match)
            if l==0:
                return 'not found'
            elif l==1:
                return match[0]
            else:
                return 'multiple results'
            """

In [12]:
#example
check_zip(85044,'AZ', 'Phoenix'),check_zip('M4M 3A6','ON','Toronto'),check_zip(70567,'BW','Stuttgart')

('US', 'CA', 'DE')

In [13]:
country=[]
for index, row in miss.iterrows():
    country.append(check_zip(row['postal_code'], row['state'], row['city']))

In [14]:
miss['country']=country
data=data.append(miss)

data['intlat2']  = [str(int(float(x))) for x in data['latitude']]
data['longlat2'] = [str(int(float(x))) for x in data['longitude']]

data['coord2']=data['intlat2']+'$'+data['longlat2']


miss=data[data['country']==''].copy()
data=data[data['country']!=''].copy()

keys=data['coord2']
values=data['country']
check=dict(zip(keys,values))

print('Fehlende: '+str(len(miss)))

Fehlende: 13


In [15]:
country=[]
for c in miss['coord2']:
    try:
        country.append(check[c])
    except:
        country.append('')

In [16]:
miss['country']=country
data=data.append(miss)

data['intlat2']  = [str(int(float(x))) for x in data['latitude']]
data['longlat2'] = [str(int(float(x))) for x in data['longitude']]

data['coord2']=data['intlat2']+'$'+data['longlat2']


miss=data[data['country']==''].copy()
data=data[data['country']!=''].copy()

keys=data['coord2']
values=data['country']
check=dict(zip(keys,values))

print('Fehlende: '+str(len(miss)))

Fehlende: 6


In [17]:
miss[columns.tolist()+['country']]

,business_id,name,city,state,postal_code,latitude,longitude,categories,country
17025,W2iWAyXd48iRkCKWOmZZuw,Viñamar,Casablanca,VS,2480000,-33.35315,-71.35729,"['Food', 'Wineries', 'Arts & Entertainment']",
19121,op_1wVF_vxbjnp_gBW9d1Q,Bodegas RE,Casablanca,VS,2340000,-33.2269211,-71.4679337,"['Arts & Entertainment', 'Wineries', 'Food']",
90421,1yQUqh3_h1IOrXZmb4CBFw,TriBeCa,Edinburgh,EDH,EH10 4HG,89.9993137156,-142.466649843,"['Restaurants', 'Breakfast & Brunch', 'America...",
124753,MImmd7Y7K3kCAP0YG05hOQ,Macerado Restaurante,Casablanca,VS,2480000,-33.3276176,-71.3949051,['Restaurants'],
154225,vBxK_MAGuy8eWL_CCfUCUQ,Lone Star,George Town,CHE,CW12 4EU,19.3276807056,-81.3810868263,"['Nightlife', 'Bars', 'Restaurants', 'Tex-Mex'...",
159672,6S3sGf8PC5dymj7VI37cuA,Al Settimo Arte - La Carpe,Saint-sauveur,WHT,7912,50.70827,3.59812,"['Pizza', 'Italian', 'Restaurants']",


In [18]:
data=data[columns.tolist()+['country']]
data[:3] 

,business_id,name,city,state,postal_code,latitude,longitude,categories,country
0,FYWN1wneV18bWNgQjJ2GNg,Dental by Design,Ahwatukee,AZ,85044,33.3306902,-111.9785992,"['Dentists', 'General Dentistry', 'Health & Me...",US
1,He-G7vWjzVUysIKrfNbPUQ,Stephen Szabo Salon,McMurray,PA,15317,40.2916853,-80.1048999,"['Hair Stylists', 'Hair Salons', ""Men's Hair S...",US
2,KQPW8lFf1y5BT2MxiSZ3QA,Western Motor Vehicle,Phoenix,AZ,85017,33.5249025,-112.1153098,"['Departments of Motor Vehicles', 'Public Serv...",US


In [19]:
data['country'].value_counts()

US    130327
CA     38378
GB      4683
DE      1111
AR        28
NO        10
AT         4
ES         2
MD         1
MX         1
IT         1
FR         1
Name: country, dtype: int64